In [1]:
from google.cloud import bigquery
import pandas as pd
import google
import os

# Establece la variable de entorno GOOGLE_APPLICATION_CREDENTIALS con la ruta al archivo de credenciales
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/raul.raggio/AppData/Roaming/gcloud/application_default_credentials.json"
credentials, project = google.auth.default()

project="peya-chile"


client = bigquery.Client(project=project, credentials=credentials)
print("Client creating using default project: {}".format(client.project))

Client creating using default project: peya-chile


c:\Users\raul.raggio\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
flag=1

In [5]:

if flag ==0:
    
    sql1 = """

       


    SELECT
    logistic_orders.country.country_name  AS logistic_orders_country__country_name,
        (FORMAT_DATE('%F', DATE_TRUNC(logistic_orders.created_date_local , WEEK(MONDAY)))) AS logistic_orders_created_date_local_week,
(logistic_orders.created_date_local ) AS logistic_orders_created_date_local_date,
        
    dim_partner.partner_name  AS dim_partner_partner_name,
    dim_partner.partner_id  AS dim_partner_partner_id,
logistic_orders.peya_order_id,
logistic_orders.order_status,
    
    (FORMAT_DATE('%A', logistic_orders.created_date_local )) AS logistic_orders_created_date_local_day_of_week,
    (EXTRACT(HOUR FROM logistic_orders.created_at_local )) AS logistic_orders_created_at_local_hour_of_day,
    (EXTRACT(MONTH FROM logistic_orders.created_at_local )) AS logistic_orders_created_at_local_month,
    dim_partner.main_cousine_category_name as cuisine,
    dim_partner.franchise.franchise_name as franchise_name,
    dim_partner.business_type.business_type_name,
    logistic_orders.city.city_name as city,
     fact_logistic_orders__zone.zone_name  AS zone,



logistic_orders.is_vendor_late_10 as is_vendor_late_10,
logistic_orders.is_vendor_late_nn as is_vendor_late_nn,

 logistic_orders.is_rider_late_10 as is_rider_late_10,

logistic_orders.is_rider_late_nn as is_rider_late_nn,




logistic_orders.estimated_prep_time/60 as estimated_prep_time,


CASE WHEN ( logistic_orders.timings.actual_delivery_time*1.000/60  ) > 45 THEN 1 ELSE 0 END
  AS slow_delivery,

CASE WHEN ( fact_orders.promisedDeliveryTime.maxMinutes   + 10) < ( logistic_orders.timings.actual_delivery_time*1.000/60  )  THEN 1  ELSE 0 END AS lcf,


 
logistic_orders.timings.at_vendor_time_cleaned/60 at_vendor_time_cleaned,
logistic_orders.timings.at_vendor_time/60 at_vendor_time,
logistic_orders.timings.assumed_actual_preparation_time/60 as assumed_prep_time,
logistic_orders.timings.promised_delivery_time/60 as promised_delivery_time,

logistic_orders.timings.actual_delivery_time/60 as actual_delivery_time,
 

logistic_orders.timings.vendor_late/60 as vendor_late_min,
logistic_orders.timings.rider_late/60 as rider_late_min,
logistic_orders.timings.hold_back_time/60 as hold_back_time_min,
logistic_orders.timings.estimated_walk_in_duration/60 as estimated_walk_in_min,
logistic_orders.timings.estimated_walk_out_duration/60 as estimated_walk_out_min,

logistic_orders.timings.to_customer_time/60 as to_customer_time_min,
  
ROUND(DATETIME_DIFF(rider.food_is_ready_at,rider.sent_to_vendor_at, SECOND)/60,1) AS RPT,
CASE WHEN rider.food_is_ready_at IS NOT NULL THEN 1 ELSE 0 END AS orders_with_fir,
  ROUND(rider.estimated_prep_time/60,1) AS EPT,
  ROUND(rider.estimated_prep_buffer/60,1) AS buffer,


FROM `peya-bi-tools-pro.il_logistics.fact_logistic_orders` AS logistic_orders
LEFT JOIN `peya-bi-tools-pro.il_core.fact_orders`  AS fact_orders ON fact_orders.order_id= logistic_orders.peya_order_id and fact_orders.registered_date = logistic_orders.created_date_local

LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS dim_partner ON dim_partner.partner_id=fact_orders.restaurant.id
LEFT JOIN UNNEST (logistic_orders.deliveries) as fact_logistic_orders__deliveries
LEFT JOIN UNNEST([logistic_orders.zone]) as fact_logistic_orders__zone

LEFT JOIN  `peya-data-origins-pro.cl_hurrier.orders_v2` ov2 on SAFE_CAST(ov2.global_order_id AS INT)=logistic_orders.peya_order_id


  WHERE ((logistic_orders.created_date_local) between  DATE_SUB(current_date(), INTERVAL 3 WEEK)  and current_date() )

AND UPPER( logistic_orders.country.country_name  ) = UPPER('chile')
AND (NOT (logistic_orders.is_preorder ) OR (logistic_orders.is_preorder ) IS NULL)


 AND ((fact_orders.restaurant.id ) > 0 OR (fact_orders.restaurant.id ) IS NULL) 
 AND (logistic_orders.peya_order_id ) IS NOT NULL



ORDER BY 15 DESC

        """
    data1=client.query(sql1).to_dataframe()
    data1.to_csv("data1.csv")
else:
    
    data1=pd.read_csv("data1.csv")

In [6]:
data1

,Unnamed: 0,logistic_orders_country__country_name,logistic_orders_created_date_local_week,logistic_orders_created_date_local_date,dim_partner_partner_name,dim_partner_partner_id,peya_order_id,order_status,logistic_orders_created_date_local_day_of_week,logistic_orders_created_at_local_hour_of_day,...,vendor_late_min,rider_late_min,hold_back_time_min,estimated_walk_in_min,estimated_walk_out_min,to_customer_time_min,RPT,orders_with_fir,EPT,buffer
0,0,Chile,2025-03-03,2025-03-07,China Wok Parque Arauco,52046.0,1477565155,completed,Friday,20,...,3.983333,-5.916667,0.783333,8.000,8.000,15.416667,NaN,0,20.0,2.0
1,1,Chile,2025-02-24,2025-03-02,PedidosYa Market - Bilbao,191688.0,1471657706,completed,Sunday,20,...,6.483333,-2.483333,0.000000,2.000,2.217,7.666667,9.2,1,7.0,0.0
2,2,Chile,2025-02-24,2025-02-26,China Wok Parque Arauco,52046.0,1466060505,cancelled,Wednesday,14,...,NaN,NaN,0.000000,8.000,8.000,0.000000,NaN,0,29.0,2.0
3,3,Chile,2025-02-24,2025-02-27,Cinnabon Costanera Center,37539.0,1467460407,completed,Thursday,16,...,19.283333,7.600000,0.983333,4.100,1.267,22.216667,11.1,1,15.0,2.0
4,4,Chile,2025-03-03,2025-03-03,Dominó - Foodtruck,159975.0,1472577059,completed,Monday,17,...,2.183333,-0.100000,5.450000,3.117,2.617,12.516667,NaN,0,11.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1868646,1868646,Chile,2025-02-17,2025-02-20,Churrascon 24/7,259095.0,1459555596,cancelled,Thursday,21,...,NaN,44.750000,0.333333,0.600,2.300,0.000000,NaN,0,12.0,1.0
1868647,1868647,Chile,2025-02-24,2025-03-02,Churrascon 24/7,259095.0,1471031619,completed,Sunday,12,...,0.300000,-4.383333,105.200000,0.967,2.450,14.450000,NaN,0,10.0,0.0
1868648,1868648,Chile,2025-02-24,2025-03-01,Churrascon 24/7,259095.0,1470041905,cancelled,Saturday,17,...,NaN,NaN,0.000000,0.967,2.450,0.000000,NaN,0,11.0,0.0
1868649,1868649,Chile,2025-03-03,2025-03-03,Churrascon 24/7,259095.0,1472112719,completed,Monday,12,...,NaN,19.633333,1.000000,0.967,2.450,12.700000,NaN,0,9.0,0.0


In [7]:
df1=data1

In [8]:
df1.columns

Index(['Unnamed: 0', 'logistic_orders_country__country_name',
       'logistic_orders_created_date_local_week',
       'logistic_orders_created_date_local_date', 'dim_partner_partner_name',
       'dim_partner_partner_id', 'peya_order_id', 'order_status',
       'logistic_orders_created_date_local_day_of_week',
       'logistic_orders_created_at_local_hour_of_day',
       'logistic_orders_created_at_local_month', 'cuisine', 'franchise_name',
       'business_type_name', 'city', 'zone', 'is_vendor_late_10',
       'is_vendor_late_nn', 'is_rider_late_10', 'is_rider_late_nn',
       'estimated_prep_time', 'slow_delivery', 'lcf', 'at_vendor_time_cleaned',
       'at_vendor_time', 'assumed_prep_time', 'promised_delivery_time',
       'actual_delivery_time', 'vendor_late_min', 'rider_late_min',
       'hold_back_time_min', 'estimated_walk_in_min', 'estimated_walk_out_min',
       'to_customer_time_min', 'RPT', 'orders_with_fir', 'EPT', 'buffer'],
      dtype='object')

In [9]:
df1["logistic_orders_created_date_local_week"].min()

'2025-02-17'

In [10]:
    df1=data1
    df1['logistic_orders_created_date_local_date'] = pd.to_datetime(df1['logistic_orders_created_date_local_date'])

    df1['franchise_name'] = df1['franchise_name'].str.lower()
    df1['cuisine'].fillna("no-cuisine", inplace=True)
    df1['city'].fillna("no city", inplace=True)
    df1['zone'].fillna("no zone", inplace=True)
    df1['franchise_name'].fillna("no franchise", inplace=True)
    df1['business_type_name'].fillna("no business", inplace=True)
    df1["month"] = df1["logistic_orders_created_date_local_date"].dt.month
    df1['logistic_orders_created_date_local_week'] = pd.to_datetime(df1['logistic_orders_created_date_local_week'])
    df1['ept_atvc'] = df1['estimated_prep_time'] + df1['at_vendor_time_cleaned']


C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_26900\3743998716.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['cuisine'].fillna("no-cuisine", inplace=True)
C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_26900\3743998716.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [11]:
df1["collection_time"] = df1["at_vendor_time"] - df1["estimated_walk_in_min"]
df1["rider_arrival"] = df1["rider_late_min"] + df1["estimated_walk_in_min"]
df1 = df1.dropna(subset=['rider_arrival', 'collection_time'])

In [24]:
#partners cambios performance
dfaux=df1[df1["business_type_name"]=="Restaurant"]
info_partner=dfaux.groupby(["dim_partner_partner_id","dim_partner_partner_name"]).agg({"peya_order_id":"nunique","is_vendor_late_10":"sum","is_vendor_late_nn":"sum","is_rider_late_10":"sum","is_rider_late_nn":"sum",'estimated_prep_time':'mean','at_vendor_time_cleaned':'mean'}).reset_index()
info_partner["%vendor_late"]=info_partner["is_vendor_late_10"]/info_partner["is_vendor_late_nn"]
info_partner["%rider_late"]=info_partner["is_rider_late_10"]/info_partner["is_rider_late_nn"]
info_partner["%_total_orders"]=info_partner["peya_order_id"]/info_partner["peya_order_id"].sum()

cambiostotal1=info_partner[(info_partner["%vendor_late"]<=0.08) & (info_partner["peya_order_id"]>=90) ]

cambiostotal1["new_ept"]=round(cambiostotal1["estimated_prep_time"]-(6-cambiostotal1["at_vendor_time_cleaned"]))
cambiostotal1.loc[cambiostotal1["new_ept"] < 5, "new_ept"] = 5
cambiostotal1

C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_26900\629461857.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cambiostotal1["new_ept"]=round(cambiostotal1["estimated_prep_time"]-(6-cambiostotal1["at_vendor_time_cleaned"]))


,dim_partner_partner_id,dim_partner_partner_name,peya_order_id,is_vendor_late_10,is_vendor_late_nn,is_rider_late_10,is_rider_late_nn,estimated_prep_time,at_vendor_time_cleaned,%vendor_late,%rider_late,%_total_orders,new_ept
1,828.0,Pita House,156,5,152,3,155,11.673077,2.042053,0.032895,0.019355,0.000105,8.0
12,5182.0,Il Romano,161,5,139,20,159,15.722222,1.892262,0.035971,0.125786,0.000108,12.0
14,5372.0,Jhot Pizza El Bosque,115,3,95,16,111,14.695652,1.462715,0.031579,0.144144,0.000077,10.0
17,5375.0,Jhot Pizza San Bernardo,267,15,248,16,264,14.550562,2.346988,0.060484,0.060606,0.000180,11.0
23,6212.0,Fuente Suiza Ñuñoa,1308,84,1258,45,1303,15.380443,2.383479,0.066773,0.034536,0.000880,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11383,537375.0,My Pizza 923,90,2,88,1,89,14.155556,2.155056,0.022727,0.011236,0.000061,10.0
11390,537545.0,Umai Sushi - Temuco,245,5,232,13,245,13.628571,1.944397,0.021552,0.053061,0.000165,10.0
11395,537639.0,Chucao Burger Sandwich,200,11,190,17,207,15.086538,2.418586,0.057895,0.082126,0.000135,12.0
11412,538038.0,Mao Sushi - La Florida,198,9,189,8,197,15.202020,1.295965,0.047619,0.040609,0.000133,10.0


In [25]:
cambiostotal1["%_total_orders"].sum()

np.float64(0.19151475438951748)

In [13]:

#ver histogramas
cambiostotal2=info_partner[(info_partner["%vendor_late"]>0.08) & (info_partner["%vendor_late"]<=0.15) & (info_partner["peya_order_id"]>=90) ]
idpartners=cambiostotal2["dim_partner_partner_id"].values
idpartners
solucion_eje_x=5

limite_rider_arrival=10
limite_collection_time=25


In [14]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit
# Inicializar una lista para guardar los resultados combinados
results_combined = []

# Iterar sobre cada id en el array idspartnerssospechosos
for partner_id_estudio in idpartners:
    dfchart = df1[
        (df1["dim_partner_partner_id"] == partner_id_estudio) &
        (df1["rider_arrival"] <= limite_rider_arrival) &
        (df1["rider_arrival"] >= -limite_rider_arrival) &  
        (df1["collection_time"] >= -limite_collection_time) & 
        (df1["collection_time"] <= limite_collection_time)
    ]
    #print(partner_id_estudio)

    if len(dfchart) == 0:
        continue

    partner_name = dfchart["dim_partner_partner_name"].iloc[0]

    # Definir la función lineal por tramos
    def piecewise_linear(x, x0, y0, k1, k2):
        return np.where(x < x0, k1 * (x - x0) + y0, k2 * (x - x0) + y0)

    bounds = ([-solucion_eje_x, -np.inf, -np.inf, -np.inf], [solucion_eje_x, np.inf, np.inf, np.inf])
    
    # Ajustar el modelo a los datos "despues"
    p0 = [0, 0, -1, 0]
    popt, _ = curve_fit(piecewise_linear, dfchart['rider_arrival'], dfchart['collection_time'], p0, bounds=bounds, maxfev=10000)

    x_offset = popt[0]
    y_offset = popt[1]
    
    # Calcular el porcentaje de puntos después del cambio de pendiente
    total_points = len(dfchart)
    points_after = len(dfchart[dfchart['rider_arrival'] > x_offset])
    points_after_abajo = len(dfchart[(dfchart['rider_arrival'] > x_offset) & (dfchart['collection_time'] <= y_offset)])
    percentage_after = (points_after / total_points) * 100

    # Guardar los parámetros ajustados
    results_combined.append({
        'partner_id': partner_id_estudio,
        'partner_name': partner_name,
        'x0': popt[0],
        'y0': popt[1],
        'k1': popt[2],
        'k2': popt[3],
        '%_despues': percentage_after,
    })

results_df = pd.DataFrame(results_combined)

In [15]:
df1.columns

Index(['Unnamed: 0', 'logistic_orders_country__country_name',
       'logistic_orders_created_date_local_week',
       'logistic_orders_created_date_local_date', 'dim_partner_partner_name',
       'dim_partner_partner_id', 'peya_order_id', 'order_status',
       'logistic_orders_created_date_local_day_of_week',
       'logistic_orders_created_at_local_hour_of_day',
       'logistic_orders_created_at_local_month', 'cuisine', 'franchise_name',
       'business_type_name', 'city', 'zone', 'is_vendor_late_10',
       'is_vendor_late_nn', 'is_rider_late_10', 'is_rider_late_nn',
       'estimated_prep_time', 'slow_delivery', 'lcf', 'at_vendor_time_cleaned',
       'at_vendor_time', 'assumed_prep_time', 'promised_delivery_time',
       'actual_delivery_time', 'vendor_late_min', 'rider_late_min',
       'hold_back_time_min', 'estimated_walk_in_min', 'estimated_walk_out_min',
       'to_customer_time_min', 'RPT', 'orders_with_fir', 'EPT', 'buffer',
       'month', 'ept_atvc', 'collection_time',

In [16]:
results_preliminar=results_df[(results_df["x0"]<=-3) & (results_df["k1"]<-1) & (results_df["k2"]<=0)  & (results_df["k2"]>=results_df["k1"])  & (results_df["%_despues"]<=80)   & (results_df["%_despues"]>=20)  ] 
info_partner=df1[(df1["dim_partner_partner_id"].isin(results_preliminar["partner_id"].values))].groupby(["dim_partner_partner_id","dim_partner_partner_name"]).agg({"peya_order_id":"nunique","is_vendor_late_10":"sum","is_vendor_late_nn":"sum","is_rider_late_10":"sum","is_rider_late_nn":"sum","estimated_prep_time":"mean"}).reset_index()
info_partner["%vendor_late"]=info_partner["is_vendor_late_10"]/info_partner["is_vendor_late_nn"]
info_partner["%rider_late"]=info_partner["is_rider_late_10"]/info_partner["is_rider_late_nn"]
info_partner_total=info_partner.merge(results_preliminar,how="left",left_on="dim_partner_partner_id",right_on="partner_id")
info_partner_total=info_partner_total.sort_values("peya_order_id",ascending=False)
cambiostotal2=info_partner_total
cambiostotal2["new_ept"]=round(cambiostotal2["estimated_prep_time"]+cambiostotal2["x0"])
cambiostotal2.loc[cambiostotal2["new_ept"] < 5, "new_ept"] = 5
cambiostotal2

,dim_partner_partner_id,dim_partner_partner_name,peya_order_id,is_vendor_late_10,is_vendor_late_nn,is_rider_late_10,is_rider_late_nn,estimated_prep_time,%vendor_late,%rider_late,partner_id,partner_name,x0,y0,k1,k2,%_despues,new_ept
18,191723.0,Oriente El Milagro,1163,149,1118,41,1159,25.243336,0.133274,0.035375,191723.0,Oriente El Milagro,-3.166615,4.490404,-1.199066,-0.281516,68.469294,22.0
25,477691.0,Mcdonald's Talca Colin,981,94,971,6,977,10.155963,0.096807,0.006141,477691.0,Mcdonald's Talca Colin,-5.000000,5.722198,-1.836449,-0.354452,71.517672,5.0
20,263727.0,Papa John's Pizza - Calama Sur,967,104,935,19,954,26.324716,0.111230,0.019916,263727.0,Papa John's Pizza - Calama Sur,-4.616664,4.336143,-1.375270,-0.141599,77.408994,22.0
29,521503.0,McDonald´s - Melipilla,955,101,934,10,944,12.692469,0.108137,0.010593,521503.0,McDonald´s - Melipilla,-4.716331,4.770021,-2.981311,-0.286897,45.680346,8.0
24,370667.0,Alemán Experto Chicureo,740,67,693,45,738,20.227763,0.096681,0.060976,370667.0,Alemán Experto Chicureo,-3.276745,2.555728,-1.037472,-0.213039,58.751903,17.0
19,228781.0,Cuatro Estaciones Pizzeria,626,67,588,31,619,14.408946,0.113946,0.050081,228781.0,Cuatro Estaciones Pizzeria,-3.033327,4.504249,-1.243740,-0.308284,66.024518,11.0
4,88007.0,Miguelayo - Macul,626,57,581,40,621,11.057052,0.098107,0.064412,88007.0,Miguelayo - Macul,-3.866324,3.320692,-1.285279,-0.200502,77.312391,7.0
21,270132.0,Punto G - Sandwich Y Papas Fritas,620,61,585,33,618,18.161290,0.104274,0.053398,270132.0,Punto G - Sandwich Y Papas Fritas,-3.304031,3.407662,-1.614813,-0.190628,66.551724,15.0
12,133607.0,Papa John's Pizza - Puente Alto,517,67,478,32,510,18.545455,0.140167,0.062745,133607.0,Papa John's Pizza - Puente Alto,-4.366276,3.976233,-1.177423,-0.204023,69.956140,14.0
16,166477.0,Johnny Rockets - Puente Alto,458,47,426,29,455,19.842795,0.110329,0.063736,166477.0,Johnny Rockets - Puente Alto,-3.533674,3.623692,-1.158529,-0.308364,67.567568,16.0


In [17]:


df1["diff_fir"]=df1["RPT"]-df1["EPT"]
df1["Ratio"]=df1["RPT"]/df1["EPT"]
df1["ci"]=df1["EPT"]-2*df1["buffer"]
df1["fir_considerado"] = df1.apply(lambda row: 1 if row["RPT"] > row["ci"] else 0, axis=1)
dfaux=df1[(df1["business_type_name"].isin(["Restaurant","Coffee"]))  ]
grouped = dfaux.groupby(['dim_partner_partner_id','dim_partner_partner_name']).agg({'EPT':'mean','RPT': 'mean','Ratio':'mean','orders_with_fir':'sum','peya_order_id':'nunique','diff_fir':'mean','fir_considerado':'sum','is_vendor_late_10':'sum','is_vendor_late_nn':'sum'})
grouped["%vendorlate"]=grouped["is_vendor_late_10"]/grouped["is_vendor_late_nn"]
grouped["%fir"]=grouped["orders_with_fir"]/grouped["peya_order_id"]
grouped["%fir_Considerado"]=grouped["fir_considerado"]/grouped["orders_with_fir"]
prueba=pd.DataFrame(grouped[["EPT","RPT","Ratio","diff_fir","orders_with_fir","peya_order_id","%fir","%fir_Considerado",'%vendorlate']])

prueba=prueba.reset_index()
dffir=prueba.copy()
dffir

dfaux=df1[ (df1["orders_with_fir"]==1) & (df1["Ratio"]<=0.1) & (df1["business_type_name"].isin(["Restaurant","Coffee"])) ]
grouped = dfaux.groupby(['dim_partner_partner_id']).agg({'RPT':'mean','orders_with_fir':'sum'})


prueba=pd.DataFrame(grouped)
prueba=prueba.reset_index()
partnersmalfir=prueba.copy()
partnersmalfir.columns=["dim_partner_partner_id","RPT_bajo","orders_with_fir_bajo"]

dffir=dffir.merge(partnersmalfir, on="dim_partner_partner_id", how="left")
dffir["%_fir_bajo"]=dffir["orders_with_fir_bajo"]/dffir["orders_with_fir"]
dffir["%_fir_bajo"].fillna(0, inplace=True)
dffir

C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_26900\1838356809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["diff_fir"]=df1["RPT"]-df1["EPT"]
C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_26900\1838356809.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Ratio"]=df1["RPT"]/df1["EPT"]
C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_26900\1838356809.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

,dim_partner_partner_id,dim_partner_partner_name,EPT,RPT,Ratio,diff_fir,orders_with_fir,peya_order_id,%fir,%fir_Considerado,%vendorlate,RPT_bajo,orders_with_fir_bajo,%_fir_bajo
0,77.0,Subway Parque Arauco - 41767,10.230769,15.160000,1.504576,5.060000,10,39,0.256410,0.500000,0.066667,NaN,NaN,0.00000
1,828.0,Pita House,11.673077,11.840625,1.043957,0.455208,96,156,0.615385,0.656250,0.032895,NaN,NaN,0.00000
2,849.0,Pasta Pazza,23.818182,19.400000,0.718519,-7.600000,2,11,0.181818,0.500000,0.125000,NaN,NaN,0.00000
3,1031.0,Niu Sushi - Tobalaba,43.726882,36.838663,0.840891,-6.943314,344,464,0.741379,0.334302,0.190476,NaN,NaN,0.00000
4,1869.0,Tarragona Independencia Rancagua,12.398773,8.936869,0.705176,-3.664141,198,326,0.607362,0.378788,0.100000,0.925,20.0,0.10101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11598,541437.0,Papas Burritos To Go,13.000000,10.925000,0.840385,-2.075000,4,8,0.500000,0.500000,0.142857,NaN,NaN,0.00000
11599,541501.0,Smash And Crunch,13.000000,13.000000,1.000000,0.000000,1,1,1.000000,1.000000,0.000000,NaN,NaN,0.00000
11600,541529.0,Happy Protein,13.000000,16.800000,1.292308,3.800000,1,2,0.500000,1.000000,0.500000,NaN,NaN,0.00000
11601,541679.0,Restaurante Maranello,13.000000,22.000000,1.692308,9.000000,1,2,0.500000,1.000000,NaN,NaN,NaN,0.00000


In [34]:
def no_cambio(row):
 
   
    if (row["%fir"] >=0.6)   &  (row["EPT"] >=10)   &  (row["Ratio"] <0.6)    &  (row["diff_fir"] <=-3)   &  (row["peya_order_id"] >90) &  (row["%vendorlate"] <0.20)    :
    
        return 1
   
    
    else:
        return 0



dffir['fir_data_baja'] = dffir.apply(no_cambio,axis=1)
cambiostotal3=dffir[(dffir["fir_data_baja"]==1) & (~dffir["dim_partner_partner_id"].isin(cambiostotal2["dim_partner_partner_id"].values)) & (~dffir["dim_partner_partner_id"].isin(cambiostotal1["dim_partner_partner_id"].values))]
cambiostotal3["new_ept"]=round(cambiostotal3["RPT"]+1)
cambiostotal3.loc[cambiostotal3["new_ept"] < 5, "new_ept"] = 5

# Verificación adicional para ajustar new_ept
cambiostotal3.loc[cambiostotal3["EPT"] - cambiostotal3["new_ept"] >= 5, "new_ept"] = round(cambiostotal3["EPT"] - 5)

cambiostotal3

C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_26900\2988435637.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cambiostotal3["new_ept"]=round(cambiostotal3["RPT"]+1)


,dim_partner_partner_id,dim_partner_partner_name,EPT,RPT,Ratio,diff_fir,orders_with_fir,peya_order_id,%fir,%fir_Considerado,%vendorlate,RPT_bajo,orders_with_fir_bajo,%_fir_bajo,fir_data_baja,new_ept
292,52576.0,Tarragona Florida Center,10.225989,5.457895,0.548308,-4.707519,133,177,0.751412,0.300752,0.087719,0.671429,7.0,0.052632,1,6.0
418,58460.0,McDonald's Santa Rosa,21.097315,8.225659,0.398406,-12.458244,1025,1482,0.691633,0.153171,0.141288,1.249500,200.0,0.195122,1,16.0
626,62969.0,Tarragona Terminal Rancagua,14.282222,5.035162,0.357515,-9.254115,401,449,0.893096,0.142145,0.158392,0.967647,102.0,0.254364,1,9.0
628,62973.0,Tarragona - Rancagua Las Américas,14.201365,8.118028,0.564161,-6.296056,355,586,0.605802,0.346479,0.140320,0.849180,61.0,0.171831,1,9.0
774,65500.0,McDonald's Antofagasta Centro,17.661694,8.553649,0.498727,-8.145885,1288,2077,0.620125,0.199534,0.154617,0.865517,203.0,0.157609,1,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8899,486489.0,Pollo Stop Parque Arauco 62,18.789474,5.407000,0.299958,-13.443000,100,114,0.877193,0.070000,0.186916,1.254839,31.0,0.310000,1,14.0
9678,505635.0,Taco Bell Plaza Sur,19.678571,5.312844,0.276294,-14.356881,109,112,0.973214,0.000000,0.131313,NaN,NaN,0.000000,1,15.0
10249,519648.0,McDonald´s - Macul Las Torres,17.756272,8.642699,0.499138,-8.283006,1171,1638,0.714896,0.146883,0.177350,0.733600,250.0,0.213493,1,13.0
10451,523824.0,Sushi Blues - San Borja,13.944915,4.667337,0.385394,-9.573869,199,236,0.843220,0.115578,0.164179,0.782716,81.0,0.407035,1,9.0


In [19]:
cambios_totales=pd.concat([cambiostotal1,cambiostotal2,cambiostotal3])
cambios_totales

,dim_partner_partner_id,dim_partner_partner_name,peya_order_id,is_vendor_late_10,is_vendor_late_nn,is_rider_late_10,is_rider_late_nn,estimated_prep_time,at_vendor_time_cleaned,%vendor_late,...,Ratio,diff_fir,orders_with_fir,%fir,%fir_Considerado,%vendorlate,RPT_bajo,orders_with_fir_bajo,%_fir_bajo,fir_data_baja
1,828.0,Pita House,156,5.0,152.0,3.0,155.0,11.673077,2.042053,0.032895,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,5182.0,Il Romano,161,5.0,139.0,20.0,159.0,15.722222,1.892262,0.035971,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,5372.0,Jhot Pizza El Bosque,115,3.0,95.0,16.0,111.0,14.695652,1.462715,0.031579,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,5375.0,Jhot Pizza San Bernardo,267,15.0,248.0,16.0,264.0,14.550562,2.346988,0.060484,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,6212.0,Fuente Suiza Ñuñoa,1308,84.0,1258.0,45.0,1303.0,15.380443,2.383479,0.066773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8899,486489.0,Pollo Stop Parque Arauco 62,114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.299958,-13.443000,100.0,0.877193,0.070000,0.186916,1.254839,31.0,0.310000,1.0
9678,505635.0,Taco Bell Plaza Sur,112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.276294,-14.356881,109.0,0.973214,0.000000,0.131313,NaN,NaN,0.000000,1.0
10249,519648.0,McDonald´s - Macul Las Torres,1638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.499138,-8.283006,1171.0,0.714896,0.146883,0.177350,0.733600,250.0,0.213493,1.0
10451,523824.0,Sushi Blues - San Borja,236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.385394,-9.573869,199.0,0.843220,0.115578,0.164179,0.782716,81.0,0.407035,1.0


In [27]:
cambios_totales["DAY-RANGE"]="MONDAY-SUNDAY"
cambios_totales["HOUR-RANGE"]="0-23"
cambios_totales["PREPARATION-BUFFER"]="2"
cambios_totales["PREPARATION-TIME"]=cambios_totales["new_ept"]
cambios_totales["STRATEGY"]="OPS_TEMPORARY"

In [28]:
cambios_totales.to_csv("cambios_totales.csv")

In [29]:
cambios_totales

,dim_partner_partner_id,dim_partner_partner_name,peya_order_id,is_vendor_late_10,is_vendor_late_nn,is_rider_late_10,is_rider_late_nn,estimated_prep_time,at_vendor_time_cleaned,%vendor_late,...,%vendorlate,RPT_bajo,orders_with_fir_bajo,%_fir_bajo,fir_data_baja,DAY-RANGE,HOUR-RANGE,PREPARATION-BUFFER,PREPARATION-TIME,STRATEGY
1,828.0,Pita House,156,5.0,152.0,3.0,155.0,11.673077,2.042053,0.032895,...,NaN,NaN,NaN,NaN,NaN,MONDAY-SUNDAY,0-23,2,8.0,OPS_TEMPORARY
12,5182.0,Il Romano,161,5.0,139.0,20.0,159.0,15.722222,1.892262,0.035971,...,NaN,NaN,NaN,NaN,NaN,MONDAY-SUNDAY,0-23,2,12.0,OPS_TEMPORARY
14,5372.0,Jhot Pizza El Bosque,115,3.0,95.0,16.0,111.0,14.695652,1.462715,0.031579,...,NaN,NaN,NaN,NaN,NaN,MONDAY-SUNDAY,0-23,2,10.0,OPS_TEMPORARY
17,5375.0,Jhot Pizza San Bernardo,267,15.0,248.0,16.0,264.0,14.550562,2.346988,0.060484,...,NaN,NaN,NaN,NaN,NaN,MONDAY-SUNDAY,0-23,2,11.0,OPS_TEMPORARY
23,6212.0,Fuente Suiza Ñuñoa,1308,84.0,1258.0,45.0,1303.0,15.380443,2.383479,0.066773,...,NaN,NaN,NaN,NaN,NaN,MONDAY-SUNDAY,0-23,2,12.0,OPS_TEMPORARY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8899,486489.0,Pollo Stop Parque Arauco 62,114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.186916,1.254839,31.0,0.310000,1.0,MONDAY-SUNDAY,0-23,2,6.0,OPS_TEMPORARY
9678,505635.0,Taco Bell Plaza Sur,112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.131313,NaN,NaN,0.000000,1.0,MONDAY-SUNDAY,0-23,2,6.0,OPS_TEMPORARY
10249,519648.0,McDonald´s - Macul Las Torres,1638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.177350,0.733600,250.0,0.213493,1.0,MONDAY-SUNDAY,0-23,2,10.0,OPS_TEMPORARY
10451,523824.0,Sushi Blues - San Borja,236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.164179,0.782716,81.0,0.407035,1.0,MONDAY-SUNDAY,0-23,2,6.0,OPS_TEMPORARY
